In [1]:
import json
import numpy as np
import pandas as pd
import neurokit2 as nk
import plotly.graph_objects as go

from arrhytmia import Arrhytmia
from mi import MI
from tools import Tools

# Create a object instance
tool = Tools()
mi = MI()
arr = Arrhytmia()

In [29]:
filename = "ekg_data_converted.json"
filename = "ekg.json"
# filename = "/Users/mauliana/Documents/Work/GAIA/git_code/ecg-interpretation-app/ekg-6lead.json"
with open(filename, "r") as file:
    data = json.load(file)

lead_names = ["I", "V1", "V2", "V4", "V6", "II"]
record_data = []

print("Save channel:")
for record in data:
    print(f"{record["channel"]} as Lead {lead_names[record["channel"] - 1]}")
    record_data.append(record["data"])



Save channel:
1 as Lead I
2 as Lead V1
3 as Lead V2


In [30]:
for index, item in enumerate(record_data):
    print(f"Lead: {lead_names[index]}, Samples: {len(item)}")

Lead: I, Samples: 3731
Lead: V1, Samples: 2975
Lead: V2, Samples: 8000


In [ ]:
def normalize_signals(record_data, sampling_rate=320):
    """
    Normalize list of ECG signals (list of lists) so they all have the same length.
    Truncate longer signals, pad shorter ones.
    
    Args:
        record_data (list of list/array): ECG recordings for each channel
        sampling_rate (int): sampling frequency in Hz (default=320)
        
    Returns:
        np.ndarray: signals with shape (n_channels, n_samples)
    """
    # 1. Find the smallest length
    lengths = [len(rec) for rec in record_data]
    min_length = min(lengths)

    # 2. Define reference length
    reference_length = max(min_length, 10 * sampling_rate)

    normalized = []
    for rec in record_data:
        if len(rec) >= reference_length:
            # Truncate
            rec_fixed = rec[:reference_length]
        else:
            # Pad with zeros at the end
            pad_len = reference_length - len(rec)
            rec_fixed = np.concatenate([rec, np.zeros(pad_len)])
        normalized.append(rec_fixed)
    
    # Convert to numpy array
    return np.array(normalized, dtype=float)


signals = normalize_signals(record_data, sampling_rate=320)

# NeuroKit expects shape (samples, channels), so transpose
signals = signals.T
print(signals.shape)


(3200, 3)


In [32]:
# convert the record data into numpy.ndarray type
# record_signals = np.array(record_data, dtype=float)

# transpose the matrix to match with neurokit library
# signals = record_signals.T

In [33]:
# Set the sampling rate 
sampling_rate = 320
# n_leads = signals.shape[1]
n_leads = len(record_data)

# Create a dummy patient data
patient_data = pd.DataFrame([{
    'patient_id': 112,
    'age': 27,
    'sex': "Male",
    'recording_date': '2025-08-13 09:17:34'
}])


In [34]:
signal = record_data[0]
ecg_signals, info = nk.ecg_process(signal, sampling_rate=sampling_rate)
interval = arr.calculate_interval(ecg_signals, info, sampling_rate, patient_data["sex"].iloc[0])
interval

,Parameter,Value,Unit,Status
0,Average Heart Rate (HR),45.86,bpm,Abnormal
1,PR Interval,189.84,ms,Normal
2,RR Interval,1817.19,ms,Abnormal
3,QRS Duration,139.06,ms,Abnormal
4,QT Interval,414.84,ms,Normal
5,QTc (Corrected QT),336.63,ms,Normal


In [38]:
# take lead I (signals[:, 0]) as a sample
ecg_signal = signals[:, 2]

ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=sampling_rate)
_, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=sampling_rate)

_, waves = nk.ecg_delineate(ecg_cleaned, rpeaks, sampling_rate=sampling_rate, method="dwt")

time = np.arange(len(ecg_cleaned)) / sampling_rate

# Plot ECG
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=ecg_cleaned, mode='lines', name='ECG Signal'))

# Final layout
fig.update_layout(
    title="ECG signal",
    xaxis_title="Time (s)",
    yaxis_title="Amplitude (mV)",
    template="plotly_white"
)

fig.show()